# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [1]:
from rdflib import Namespace, URIRef, RDFS
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK
from buildingmotif.model_builder import TemplateBuilderContext

In [2]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
brick = Library.load(ontology_graph="Brick.ttl")

2025-07-03 14:07:45,809 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/prefix (Name: http://qudt.org/2.1/vocab/prefix). Check this is loaded into BuildingMOTIF
2025-07-03 14:07:45,810 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/unit (Name: http://qudt.org/2.1/vocab/unit). Check this is loaded into BuildingMOTIF
2025-07-03 14:07:45,810 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/collection/usertest (Name: http://qudt.org/2.1/collection/usertest). Check this is loaded into BuildingMOTIF
2025-07-03 14:07:45,811 | root |  WARNING: An ontology could not resolve a dependency on https://w3id.org/rec/recimports (Name: https://w3id.org/rec/recimports). Check this is loaded into BuildingMOTIF
2025-07-03 14:07:45,811 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/overlay/qudt (Name: http://qudt.org/2.1/schema/shacl/o

In [3]:
brick_tmpl = Library.load(directory='brick-templates')

In [4]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
# ahu = ctx['multiple-zone-ahu'](name='ahu1')
# # now create an instance of the water flow property
# flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
# flow_prop['name-owner'] = my_tank['out']
# # now create the sensor
# flow_sensor = ctx['sensor'](name='my_flow_sensor')
# flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
# flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [11]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs_per_ahu, reheat_vavs_per_ahu]
# Could add amt computer spaces_per_ahu to building if it serves example later

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(brick_tmpl)
[4,]

[4]

In [12]:
bldg_config = {'ahus': 2, 'cv': 0, 'hv': 0}
bldg_ctx_dict = {}
for i in range(bldg_config['ahus']):
    ahu_name = f'ahu_{i}'
    # bldg_ctx_dict[name] = ctx['multiple-zone-ahu'](name=name)
    ahu = ctx['multiple-zone-ahu'](name=ahu_name)
    for cv in range(bldg_config['cv']):
        cv_name = f'cvav_{i}_{cv}'
        cvav = ctx['vav-cooling-only'](name=cv_name)
        ctx['feeds'](name = ahu['name'], target = cvav['name'])
    for hv in range(bldg_config['hv']):
        hv_name = f'hvav_{i}_{hv}'
        hvav = ctx['vav-with-reheat'](name=hv_name)
        ctx['feeds'](name = ahu['name'], target = hvav['name'])

In [13]:
bldg.add_graph(ctx.compile())

In [14]:
print(bldg.graph.serialize(format='ttl'))

@prefix : <urn:example#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

: a owl:Ontology .

:ahu_0 a brick:AHU ;
    rdfs:label "AHU" ;
    brick:hasPart :clg_coil_2975c1fc,
        :clg_coil_907263d9,
        :htg_coil_8880e655,
        :htg_coil_ae981a4e,
        :ra_damper_624e6aa8,
        :ra_damper_c22e0d3a,
        :sa_fan_1592d24b,
        :sa_fan_db75c482 ;
    brick:hasPoint :oa_temp_1bab1b43,
        :oa_temp_8c33cae6,
        :sa_temp_1f895fab,
        :sa_temp_c3da4f40 .

:ahu_1 a brick:AHU ;
    rdfs:label "AHU" ;
    brick:hasPart :clg_coil_fbf5417f,
        :htg_coil_c00ba1af,
        :ra_damper_6195dca3,
        :sa_fan_c8f277ec ;
    brick:hasPoint :oa_temp_98e17afd,
        :sa_temp_b5f0cbef .

:clg_coil-valve_cmd_428411f9 a brick:Valve_Position_Command ;
    rdfs:label "Valve_Position_Command" .

:clg_coil-valve_cmd_62625e70 a brick:Valve_Position_Com

In [15]:
bldg.graph.serialize('brick-example.ttl', format = 'ttl')

<Graph identifier=d58ab3df-3f76-4331-83de-104de488454f (<class 'rdflib.graph.Graph'>)>